In [2]:
import tensorflow as tf
import os
import numpy as np


In [6]:
home = os.environ['HOME']
base_dir = os.path.join(home, 'YogaModel/dataset')
train_dir = os.path.join(base_dir, 'train')
#test_dir = os.path.join(base_dir, 'test')

In [12]:
from  tensorflow.keras.preprocessing.image import ImageDataGenerator
train_data_gen = ImageDataGenerator(
    rescale = 1.0/255,
    rotation_range = 40,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    shear_range = 0.2,
    horizontal_flip = True,
    fill_mode = 'nearest'
)
train_data = train_data_gen.flow_from_directory(
    train_dir,
    target_size=(150,150),
    class_mode='categorical',
    batch_size=20
)

Found 5991 images belonging to 107 classes.
